# Input:
1)dataset of skills

2) musthave skills: keyword -> job title

3) dataset of skills by vacancies

# Output 
Skills most likely to go together

In [ ]:
import pandas as pd
import numpy as np
#import cPickle
try:
    import cPickle 
except:
    import pickle as cPickle
from nltk.corpus import stopwords
from gensim.models import word2vec
import nltk.data
import re
import logging

In [13]:
with open("prep_data_tokens_underscore_1", "rb") as g:
    data_dice = cPickle.load(g)
print (len(data_dice))
data_dice[3705]

7146


['f#',
 'haskell',
 'erlang',
 'scala',
 'prolog',
 'sinatra',
 'riak',
 'clojure',
 'jvm',
 'elasticsearch',
 '3d_rendering',
 'apache_storm',
 'paas',
 'lisp',
 'otv',
 'virtual_private_cloud',
 'lacp',
 'vtp',
 'nexus',
 'dna_sequencing',
 'next_generation_sequencing',
 'assays',
 'bioinformatics',
 'biochemistry',
 'dna',
 'functional_programming',
 'microsoft_dynamics_sl',
 'distributed_computing',
 'test-driven_development',
 'tdd',
 'front_office']

In [20]:
data_must_have = pd.read_csv('mustHaveSkills-2.csv', header = 0, encoding='ISO-8859-1')
del data_must_have['job_brief_id']

print(len(data_must_have))
data_must_have = data_must_have.drop_duplicates(subset=['keyword_name', 'job_title'], keep='last')
data_must_have = data_must_have[data_must_have["job_title"] != 0]
print (len(data_must_have))
data_must_have[1000:1010]

261724
80810


,keyword_name,job_title
3114,user experience research,user experience research
3115,user experience research,ux researcher
3116,user experience research,user experience researcher
3117,user experience research,user research
3118,user experience research,user reseacher
3119,user experience research,User Research Coordinator
3120,ux researcher,ux research
3121,ux researcher,user experience research
3122,ux researcher,ux researcher
3123,ux researcher,user experience researcher


In [25]:
# next line fails
# data_must_have['Count'] = data_must_have.groupby('job_title')['keyword_name'].transform(pd.Series.value_counts)
data_must_have.drop_duplicates(inplace=True)
data_must_have['keyword_name'] = data_must_have['keyword_name'].str.lower()
data_must_have['keyword_name'] = data_must_have['keyword_name'].str.replace(' ' ,'_')
data_must_have['job_title'] = data_must_have['job_title'].str.lower()
#gr_df_keywordname = data_must_have.groupby('keyword_name')['job_title'].apply(list)
gr_df_jobtitle = data_must_have.groupby('job_title')['keyword_name'].apply(list)

gr_df_jobtitle

job_title
.net                             [microsoft_office_sharepoint_server]
.net application                       [architect, .net, asp.net, c#]
.net developer      [.net, sharepoint, moss, asp.net, html/html5, ...
.net engineer       [.net, asp.net, javascript, mobile_application...
.net programmer                                                [.net]
                                          ...                        
xbox security                            [c++, xbox, ps, playstation]
xd                  [ux_design, ux, uxd, ued, xd, user_experience_...
xml developer                                        [xml, html, css]
yield manager       [yield, online_media, digital, advertising, br...
zbb analysts           [zerbased_budgeting, zbb, accounting, finance]
Name: keyword_name, Length: 4596, dtype: object

In [29]:
must_have_data = []
for vector_list in gr_df_jobtitle:
    xx = list(set(vector_list))
    if xx not in must_have_data:
        must_have_data.append(xx)

len(must_have_data)
must_have_data[20]

['organic', 'games']

In [82]:
data_naruki = pd.read_csv('naukri_skill_full', header = 0, encoding='ISO-8859-1')
# drop duplicate
data_naruki.drop_duplicates(subset=['id', 'skill'], keep='last')
# lower-case
data_naruki['skill'] = data_naruki['skill'].str.lower()
data_naruki['skill'] = data_naruki['skill'].str.replace(' ','_')
data_naruki_final = data_naruki.groupby('id')['skill'].apply(list)
print (len(data_naruki_final))
data_naruki_final[9:10]

120107


id
9    [.net_support_analyst, sql_queries, .net, sql,...
Name: skill, dtype: object

In [64]:
data_train_w2v = data_dice
for must_have in gr_df_jobtitle:
    if len(must_have) > 2:
        if must_have not in data_train_w2v:
            data_train_w2v.append(must_have)
            
len(data_train_w2v)

107670

In [55]:
for skills in data_naruki_final:
    if len(skills) > 2 and skills not in data_train_w2v:
        data_train_w2v.append(skills)
        
len(data_train_w2v)
# takes time to run this boi

107670

In [72]:
with open('./duyet_data_train_w2v', 'wb') as f:
    cPickle.dump(data_train_w2v_for_check, f)

In [74]:
#b- to d in rd
with open('./duyet_data_train_w2v', 'rb') as f:
    data_train_w2v = cPickle.load(f)

In [76]:
data_train_w2v

[['addm',
  'configuration_management_database',
  'servicenow',
  'it_service_management',
  'itil',
  'service_management',
  'ddm',
  'bmc_remedy',
  'issue_tracking',
  'configuration',
  'knowledge_management',
  'help_desk',
  'normalization',
  'bppm',
  'patrol',
  'bmc_patrol',
  'event_management',
  'systems_management',
  'performance_management',
  'awr',
  'tkprof',
  'sql_trace',
  'data_pump',
  'oracle_dba',
  'oracle_grid_control',
  'statspack',
  'tde',
  'recovery',
  'mef',
  'vldb',
  'nmon'],
 ['adabas',
  'natural',
  'energy',
  'roscoe',
  'fiscal_management',
  'jcl',
  'vse',
  'entirex',
  'war',
  'jar',
  'webmethods',
  'root_cause_analysis',
  'cost_estimation',
  'assembly_language',
  'bal',
  'smp/e',
  'ipcs',
  'tpf',
  'pl/i',
  'ibm_utilities',
  'rexx',
  'syncsort',
  'clist',
  'smf',
  'panvalet',
  'tso',
  'ca-7',
  'endevor',
  'ca_ops/mvs',
  'computer_operator',
  'spf'],
 ['acsls',
  'netbackup',
  'backup',
  'backup_administration',


In [77]:
data_train_w2v_for_check = data_train_w2v
data_train_w2v = []

In [78]:


for i in data_train_w2v_for_check:
    vector =[]
    for j in i:
        # TODO: remove stopword, clean unicode becomes string
        if isinstance(j, str):
            if 1 == 1: #  or j not in vector:
                vector.append(j)
    if i not in data_train_w2v:
        data_train_w2v.append(vector)

print (len(data_train_w2v))

107609


In [68]:
import multiprocessing

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)    

print ("Training model...")
model = word2vec.Word2Vec(data_train_w2v, 
          workers=multiprocessing.cpu_count(),  # Number of threads to run in parallel
          size=300, 
          min_count=1, 
          window=10, 
          sample = 1e-3,  # Downsample setting for frequent words
          iter=4,
          sg =1
    )

model.init_sims(replace=True)


2021-01-10 14:52:25,014 : INFO : collecting all words and their counts
2021-01-10 14:52:25,016 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-01-10 14:52:25,111 : INFO : PROGRESS: at sentence #10000, processed 244238 words, keeping 13709 word types
2021-01-10 14:52:25,135 : INFO : PROGRESS: at sentence #20000, processed 330742 words, keeping 26910 word types
2021-01-10 14:52:25,162 : INFO : PROGRESS: at sentence #30000, processed 417024 words, keeping 35993 word types


Training model...


2021-01-10 14:52:25,198 : INFO : PROGRESS: at sentence #40000, processed 503576 words, keeping 44859 word types
2021-01-10 14:52:25,235 : INFO : PROGRESS: at sentence #50000, processed 592651 words, keeping 53040 word types
2021-01-10 14:52:25,271 : INFO : PROGRESS: at sentence #60000, processed 678954 words, keeping 59874 word types
2021-01-10 14:52:25,295 : INFO : PROGRESS: at sentence #70000, processed 764732 words, keeping 67222 word types
2021-01-10 14:52:25,332 : INFO : PROGRESS: at sentence #80000, processed 848896 words, keeping 74145 word types
2021-01-10 14:52:25,369 : INFO : PROGRESS: at sentence #90000, processed 938167 words, keeping 79239 word types
2021-01-10 14:52:25,392 : INFO : PROGRESS: at sentence #100000, processed 1026780 words, keeping 85048 word types
2021-01-10 14:52:25,420 : INFO : collected 89697 word types from a corpus of 1092226 raw words and 107609 sentences
2021-01-10 14:52:25,421 : INFO : Loading a fresh vocabulary
2021-01-10 14:52:25,742 : INFO : effec

In [79]:
model.wv.save_word2vec_format('duyet_word2vec_skill.bin', binary=True)

2021-01-10 15:57:09,237 : INFO : storing 89697x300 projection weights into duyet_word2vec_skill.bin


# Testing

In [80]:
model.similar_by_word('machine_learning')

C:\Users\Dima\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similar_by_word` (Method will be removed in 4.0.0, use self.wv.similar_by_word() instead).
  """Entry point for launching an IPython kernel.


[('data_science', 0.9521018266677856),
 ('artificial_intelligence', 0.9443296194076538),
 ('data_scientist', 0.9367461800575256),
 ('nlp', 0.9322383403778076),
 ('algorithm_development', 0.9259664416313171),
 ('natural_language_processing', 0.9215646386146545),
 ('weka', 0.9165726900100708),
 ('scikit-learn', 0.9148728847503662),
 ('time_series_analysis', 0.9130783081054688),
 ('pattern_recognition', 0.9124218821525574)]

In [81]:
model.similar_by_word('python')

C:\Users\Dima\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similar_by_word` (Method will be removed in 4.0.0, use self.wv.similar_by_word() instead).
  """Entry point for launching an IPython kernel.


[('ruby', 0.8728437423706055),
 ('bash', 0.844484806060791),
 ('perl', 0.8339748382568359),
 ('devops_automation_tools', 0.830797553062439),
 ('python_or_bash_or_ruby_or_perl)', 0.827045202255249),
 ('site_reliability', 0.8245991468429565),
 ('computer_programming', 0.8245022296905518),
 ('pgsql', 0.8239307999610901),
 ('shell', 0.8228707313537598),
 ('scripting_language', 0.8227330446243286)]

In [8]:
model.similar_by_word('css')

[(u'html5', 0.8974307179450989),
 (u'ui_development', 0.8635860681533813),
 (u'bootstrap', 0.8595981597900391),
 (u'css3', 0.8528745770454407),
 (u'html_5', 0.8408917784690857),
 (u'web_2.0', 0.8406175374984741),
 (u'xhtml', 0.8400400876998901),
 (u'ajax', 0.8394852876663208),
 (u'web_development', 0.8363163471221924),
 (u'angular', 0.8361284136772156)]

In [9]:
model.similar_by_word('html')

[(u'ajax', 0.8565713763237),
 (u'xhtml', 0.8516340255737305),
 (u'web_development', 0.8487673997879028),
 (u'html5', 0.8457298278808594),
 (u'css3', 0.8286351561546326),
 (u'html_5', 0.8208118677139282),
 (u'java_script', 0.8198404908180237),
 (u'dhtml', 0.8177890777587891),
 (u'css', 0.8029626607894897),
 (u'javascript', 0.8021034598350525)]

In [10]:
model.similar_by_word('html5')

[(u'css3', 0.9599962830543518),
 (u'ui_development', 0.9453151822090149),
 (u'ui_developer', 0.9411906599998474),
 (u'bootstrap', 0.9401106834411621),
 (u'angular', 0.9390008449554443),
 (u'front-end', 0.9382939338684082),
 (u'frontend', 0.9346851110458374),
 (u'jquerry', 0.9328276515007019),
 (u'user_interface_development', 0.9299989938735962),
 (u'object_oriented_javascript', 0.9298042058944702)]

In [11]:
model.similar_by_word('bootstrap')

[(u'angularjs', 0.9698102474212646),
 (u'angular', 0.9694699048995972),
 (u'backbone.js', 0.9692836999893188),
 (u'angular.js', 0.9609279632568359),
 (u'angular_js', 0.9575139284133911),
 (u'front_-_end_developer', 0.9525660276412964),
 (u'front_end_developer', 0.9521020650863647),
 (u'sass', 0.9519370198249817),
 (u'oojs', 0.9508299827575684),
 (u'backbonejs', 0.9491092562675476)]

In [12]:
model.similar_by_word('javascript')

[(u'ajax', 0.8866481781005859),
 (u'html5', 0.8692954182624817),
 (u'css3', 0.8681623935699463),
 (u'web_development', 0.8675326704978943),
 (u'java_script', 0.8644356727600098),
 (u'html_5', 0.853226363658905),
 (u'bootstrap', 0.852524995803833),
 (u'xhtml', 0.8516951203346252),
 (u'dojo', 0.8443065285682678),
 (u'jquery', 0.843605637550354)]

In [13]:
model.similar_by_word('nodejs')

[(u'node_js', 0.9847288727760315),
 (u'node.js', 0.9839949607849121),
 (u'mean_stack', 0.9811791181564331),
 (u'reactjs', 0.980007529258728),
 (u'react.js', 0.9778978824615479),
 (u'backbonejs', 0.9751937985420227),
 (u'expressjs', 0.9747955799102783),
 (u'grunt', 0.9732896089553833),
 (u'node_java_script', 0.9731259346008301),
 (u'knockoutjs', 0.9729543924331665)]

In [14]:
model.similar_by_word('node.js')

[(u'nodejs', 0.9839949607849121),
 (u'angular.js', 0.9655287265777588),
 (u'react.js', 0.9631706476211548),
 (u'node_js', 0.9619911909103394),
 (u'mean_stack', 0.9599627256393433),
 (u'ember.js', 0.9583311676979065),
 (u'angularjs', 0.9580939412117004),
 (u'java-script', 0.9555432200431824),
 (u'backbone.js', 0.9541488885879517),
 (u'reactjs', 0.954147219657898)]

In [15]:
model.similar_by_word('php')

[(u'drupal', 0.9372311234474182),
 (u'wordpress', 0.9317398071289062),
 (u'core_php', 0.928536593914032),
 (u'zend', 0.9279359579086304),
 (u'cakephp', 0.9255720973014832),
 (u'lamp', 0.925187885761261),
 (u'joomla', 0.924292266368866),
 (u'magento', 0.9184141159057617),
 (u'codeigniter', 0.9148786067962646),
 (u'oscommerce', 0.910626232624054)]

In [16]:
model.similar_by_word('c++')

[(u'c', 0.9312055110931396),
 (u'programming', 0.8374342918395996),
 (u'languages', 0.8336395025253296),
 (u'embedded_linux', 0.8176031112670898),
 (u'rtos', 0.8171031475067139),
 (u'c++_programming', 0.8134716153144836),
 (u'stl', 0.8129345774650574),
 (u'boost', 0.8118593096733093),
 (u'vc++', 0.8098419308662415),
 (u'embedded_systems', 0.8087258338928223)]

In [17]:
model.similar_by_word('web')

[(u'websites', 0.9125840663909912),
 (u'interactive', 0.8940144777297974),
 (u'html_and_css', 0.8812004327774048),
 (u'producer', 0.8783373236656189),
 (u'website_designing', 0.8759322166442871),
 (u'html_xhtml', 0.8700681328773499),
 (u'web_designer', 0.8636655211448669),
 (u'dream_weaver', 0.8634325265884399),
 (u'visual', 0.8630934953689575),
 (u'search_engine_ranking', 0.8610385656356812)]

In [18]:
model.similar_by_word('rails')

[(u'ror', 0.9833118319511414),
 (u'jruby', 0.9804388284683228),
 (u'ruby_on_rails', 0.980139970779419),
 (u'opensource', 0.9721523523330688),
 (u'go', 0.9704596996307373),
 (u'pyramid', 0.968437135219574),
 (u'ruby_rails_developer', 0.9672976732254028),
 (u'lamp_stack', 0.9660986065864563),
 (u'content_delivery_network', 0.9656563997268677),
 (u'twisted', 0.9646377563476562)]

In [19]:
model.similar_by_word('ruby')

[(u'bash', 0.9099234938621521),
 (u'ruby_on_rails', 0.9087743759155273),
 (u'rails', 0.9037313461303711),
 (u'technoology_sales', 0.9015157222747803),
 (u'ror', 0.8960175514221191),
 (u'leading_teams', 0.8955328464508057),
 (u'jruby', 0.8951069712638855),
 (u'shell', 0.894086480140686),
 (u'django', 0.890491783618927),
 (u'site_reliability', 0.8903191089630127)]

In [20]:
model.similar_by_word('mysql')

[(u'postgresql', 0.8904533386230469),
 (u'zend', 0.8901103138923645),
 (u'php', 0.886795163154602),
 (u'apache', 0.8820366859436035),
 (u'cakephp', 0.8752923011779785),
 (u'my_sql', 0.8735089302062988),
 (u'lamp', 0.8731725215911865),
 (u'open_source', 0.8700281381607056),
 (u'zencart', 0.8655450344085693),
 (u'zend_framework', 0.8649144172668457)]

In [21]:
model.similar_by_word('db2')

[(u'mainframe', 0.9391623735427856),
 (u'jcl', 0.9366923570632935),
 (u'as_400', 0.9339416027069092),
 (u'cics', 0.9314805865287781),
 (u'rpg', 0.9302469491958618),
 (u'as400', 0.9189743995666504),
 (u'cobol', 0.916329026222229),
 (u'vsam', 0.9148311614990234),
 (u'ims_dc', 0.9146862030029297),
 (u'iseries', 0.912138819694519)]

In [22]:
model.similar_by_word('sql')

[(u'sql_server', 0.8586803674697876),
 (u'sql_queries', 0.8171682357788086),
 (u'ms_sql', 0.8151894807815552),
 (u'rdbms', 0.7995103001594543),
 (u'oracle_sql', 0.7978106141090393),
 (u'sql_database', 0.7919921875),
 (u'ms_sql_server', 0.7887479066848755),
 (u'sql_development', 0.7685669660568237),
 (u'oracle', 0.7604414224624634),
 (u'stored_procedures', 0.7512783408164978)]

In [23]:
model.similar_by_word('mssql')

[(u'sql_2008', 0.977550208568573),
 (u'database_programming', 0.9753374457359314),
 (u't_sql', 0.971894383430481),
 (u't/sql', 0.9690109491348267),
 (u't-sql', 0.9673973321914673),
 (u'microsoft_sql', 0.9663206338882446),
 (u'sql_programming', 0.9655624628067017),
 (u'ms_-_sql', 0.9621620178222656),
 (u'tsql', 0.9614934325218201),
 (u'microsoft_sql_server', 0.961333155632019)]

In [24]:
model.similar_by_word('db2')

[(u'mainframe', 0.9391623735427856),
 (u'jcl', 0.9366923570632935),
 (u'as_400', 0.9339416027069092),
 (u'cics', 0.9314805865287781),
 (u'rpg', 0.9302469491958618),
 (u'as400', 0.9189743995666504),
 (u'cobol', 0.916329026222229),
 (u'vsam', 0.9148311614990234),
 (u'ims_dc', 0.9146862030029297),
 (u'iseries', 0.912138819694519)]

In [25]:
model.similar_by_word('html5')

[(u'css3', 0.9599962830543518),
 (u'ui_development', 0.9453151822090149),
 (u'ui_developer', 0.9411906599998474),
 (u'bootstrap', 0.9401106834411621),
 (u'angular', 0.9390008449554443),
 (u'front-end', 0.9382939338684082),
 (u'frontend', 0.9346851110458374),
 (u'jquerry', 0.9328276515007019),
 (u'user_interface_development', 0.9299989938735962),
 (u'object_oriented_javascript', 0.9298042058944702)]

In [26]:
model.similar_by_word('oracle')

[(u'oracle_sql', 0.820723295211792),
 (u'oracle_database', 0.8060543537139893),
 (u'web_sphere', 0.7909789085388184),
 (u'plsql', 0.7906963229179382),
 (u'pl', 0.7901702523231506),
 (u'rdbms', 0.786513090133667),
 (u'system_analyst', 0.7857550382614136),
 (u'unix', 0.7848482131958008),
 (u'oracle_rdbms', 0.7839452624320984),
 (u'sql_databases', 0.78342604637146)]

In [27]:
model.similar_by_word('php5')

[(u'phalcon', 0.9917563796043396),
 (u'webdeveloper', 0.9917334318161011),
 (u'.web_developer', 0.9917197823524475),
 (u'php_development', 0.991561770439148),
 (u'php_5', 0.9905673265457153),
 (u'.web_software_engineer', 0.9891971945762634),
 (u'symfony_framework', 0.9889212250709534),
 (u'php_architect', 0.9887924194335938),
 (u'php_&amp;_mysql', 0.9883192777633667),
 (u'senior_php_developer', 0.9883060455322266)]

In [28]:
model.similar_by_word('asp')

[(u'classic_asp', 0.9644899368286133),
 (u'microsoft_visual_studio', 0.961052656173706),
 (u'visual_basic_.net', 0.960399329662323),
 (u'ms_sql_server_2005', 0.9572274684906006),
 (u'user_controls', 0.9556883573532104),
 (u'windows_sharepoint_services', 0.9555542469024658),
 (u'dot_net_framework', 0.9541379809379578),
 (u'com+', 0.9538134932518005),
 (u'remoting', 0.953339695930481),
 (u'vs.net', 0.9508565068244934)]

In [29]:
model.similar_by_word('svm')

[(u'boosting', 0.9351357817649841),
 (u'theano', 0.9252413511276245),
 (u'h2o', 0.9208034873008728),
 (u'random_forest', 0.916857123374939),
 (u'machine_training', 0.9107584357261658),
 (u'caffe', 0.9083511233329773),
 (u'apache_sqoop', 0.908349335193634),
 (u'neural_networks', 0.908269464969635),
 (u'octave', 0.9066129922866821),
 (u'text_analytics', 0.906141459941864)]

In [30]:
model.similar_by_word('django')

[(u'flask', 0.9536893367767334),
 (u'python_developer', 0.950721025466919),
 (u'ror', 0.9489517211914062),
 (u'twisted', 0.9473798871040344),
 (u'lamp_stack', 0.9445514678955078),
 (u'pyramid', 0.9436725378036499),
 (u'pytohn', 0.9436454772949219),
 (u'backend_development', 0.9435844421386719),
 (u'mean_stack_development', 0.9435287714004517),
 (u'api_design', 0.9434955716133118)]

In [31]:
model.similar_by_word('mongodb')

[(u'nosql', 0.9644761085510254),
 (u'node.js', 0.9415790438652039),
 (u'postgres', 0.9353598356246948),
 (u'full_stack', 0.9341529011726379),
 (u'solr', 0.9325692653656006),
 (u'mongo_db', 0.9288532137870789),
 (u'nodejs', 0.9269697666168213),
 (u'redis', 0.926455020904541),
 (u'cassandra', 0.9255849719047546),
 (u'mean_stack', 0.9224556684494019)]

In [32]:
model.similar_by_word('mongo')

[(u'elastic_search', 0.9851661920547485),
 (u'no_sql', 0.9809781312942505),
 (u'neo4j', 0.9749455451965332),
 (u'oracle_nosql_database', 0.972252368927002),
 (u'postgres', 0.9706345796585083),
 (u'high_performance', 0.9705182313919067),
 (u'google_cloud', 0.9689809679985046),
 (u'redshift', 0.9687713384628296),
 (u'python_django', 0.9683542847633362),
 (u'zeromq', 0.9679644703865051)]

In [33]:
model.similar_by_word('falcon')

[(u'changman', 0.9860578775405884),
 (u'vantage', 0.9837855100631714),
 (u'jcl_developer', 0.9836094975471497),
 (u'coolgen_developer', 0.9831973910331726),
 (u'ca7', 0.9828194379806519),
 (u'upgrading', 0.9826165437698364),
 (u'wma', 0.9819715023040771),
 (u'endevour', 0.9814755916595459),
 (u'lifecomm', 0.9801531434059143),
 (u'cobal', 0.9799065589904785)]

In [34]:
model.similar_by_word('express')

[(u'lamp_stack', 0.9035072326660156),
 (u'*_magento_developer_certification', 0.8985456228256226),
 (u'jquerry', 0.898064374923706),
 (u'sales_force_development', 0.8953841924667358),
 (u'php_fullstack_developer', 0.8941729068756104),
 (u'object_oriented_php_programming', 0.8934909105300903),
 (u'full_stack_web_development', 0.8926893472671509),
 (u'shopify', 0.8924915790557861),
 (u'saas_-less', 0.8921962380409241),
 (u'senior_web_developer', 0.8918324708938599)]

In [35]:
model.similar_by_word('spark')

[(u'hdfs', 0.975350022315979),
 (u'hive', 0.9712682962417603),
 (u'sparks', 0.9664201140403748),
 (u'cassandra', 0.9662449359893799),
 (u'storm', 0.9661303758621216),
 (u'scala', 0.9647467136383057),
 (u'map-reduce', 0.9645686745643616),
 (u'pig', 0.964016854763031),
 (u'data_engineering', 0.9635540246963501),
 (u'ambari', 0.9633440971374512)]

In [36]:
model.similar_by_word('hadoop')

[(u'hbase', 0.9534479975700378),
 (u'hive', 0.9463975429534912),
 (u'pig', 0.938024640083313),
 (u'spark', 0.936820387840271),
 (u'big_data', 0.929339587688446),
 (u'hdfs', 0.9256489276885986),
 (u'scala', 0.9244800209999084),
 (u'cassandra', 0.9214069843292236),
 (u'mapreduce', 0.9166395664215088),
 (u'bigdata', 0.9135311841964722)]

In [37]:
model.similar_by_word('hive')

[(u'hdfs', 0.9864153861999512),
 (u'spark', 0.9712682962417603),
 (u'pig', 0.9662244319915771),
 (u'hbase', 0.9647097587585449),
 (u'map_reduce', 0.9629838466644287),
 (u'cassandra', 0.9532560706138611),
 (u'hiveql', 0.9513139128684998),
 (u'ambari', 0.9501124620437622),
 (u'sqoop', 0.9487943649291992),
 (u'apache_flume', 0.9486385583877563)]

In [38]:
model.similar_by_word('impala')

[(u'storm', 0.9902775287628174),
 (u'big_data_architect', 0.9894396066665649),
 (u'oozie', 0.9876737594604492),
 (u'sqoop', 0.9868731498718262),
 (u'ambari', 0.9854146242141724),
 (u'sparks', 0.9847355484962463),
 (u'cloudera_impala', 0.9839766621589661),
 (u'hortonworks', 0.9838944673538208),
 (u'hadoop_development', 0.9831902980804443),
 (u'big_data_technologies', 0.9826507568359375)]

In [39]:
model.similar_by_word('oozie')

[(u'flume', 0.9978626370429993),
 (u'sqoop', 0.993576169013977),
 (u'hortonworks', 0.9927583932876587),
 (u'big_data_architect', 0.9914911389350891),
 (u'hadoop_development', 0.9908590316772461),
 (u'ambari', 0.9900696873664856),
 (u'storm', 0.9888783693313599),
 (u'shark', 0.9877080917358398),
 (u'impala', 0.9876737594604492),
 (u'hadoop_developer', 0.9875854253768921)]

In [41]:
model.similar_by_word('nginx')

[(u'varnish', 0.9811279773712158),
 (u'lighttpd', 0.9765704274177551),
 (u'memcached', 0.9754940271377563),
 (u'haproxy', 0.9726861715316772),
 (u'phpunit', 0.9696424007415771),
 (u'aws_elastic_beanstalk', 0.9688529372215271),
 (u'vagrant', 0.9684524536132812),
 (u'debian_linux', 0.9671319723129272),
 (u'zabbix', 0.9663083553314209),
 (u'memcache', 0.9658818244934082)]

In [42]:
model.similar_by_word('rest')

[(u'json', 0.9250497221946716),
 (u'soap', 0.9241546988487244),
 (u'restful', 0.923047661781311),
 (u'restful_web_services', 0.899715781211853),
 (u'apigee', 0.8967372179031372),
 (u'rest_api', 0.8954670429229736),
 (u'worklight', 0.8930211067199707),
 (u'rest_services', 0.8915902376174927),
 (u'android_app', 0.8899410963058472),
 (u'singleton', 0.8897507190704346)]

In [43]:
model.similar_by_word('.net')

[(u'c#', 0.9468095898628235),
 (u'asp.net', 0.9315366744995117),
 (u'c#.net', 0.9251785278320312),
 (u'dot_net', 0.9211883544921875),
 (u'dotnet', 0.9155071973800659),
 (u'.net_developer', 0.9129540324211121),
 (u'asp_.net', 0.9070920944213867),
 (u'vb.net', 0.8996402025222778),
 (u'.net_framework', 0.8986883759498596),
 (u'vb_.net', 0.8871078491210938)]

In [44]:
model.similar_by_word('perl')

[(u'shell', 0.8857336044311523),
 (u'shell_scripting', 0.8838289380073547),
 (u'bash', 0.8750884532928467),
 (u'scripting', 0.8634020686149597),
 (u'devops_automation_tools', 0.8516108393669128),
 (u'ruby', 0.8494759798049927),
 (u'tcl', 0.8479472398757935),
 (u'scripts', 0.8474525213241577),
 (u'sre', 0.8432766199111938),
 (u'perl_scripting', 0.8419920206069946)]

In [45]:
model.similar_by_word('unity')

[(u'games_console', 0.9214930534362793),
 (u'sublime_text', 0.9046214818954468),
 (u'unity3d', 0.8957223892211914),
 (u'technical_artist', 0.8919845819473267),
 (u'microsoft_direct3d', 0.8916783928871155),
 (u'vista', 0.8900236487388611),
 (u'unity_3d', 0.8878706693649292),
 (u'low-level_engine_systems', 0.8852602243423462),
 (u'console', 0.8846390247344971),
 (u'aaa', 0.8795377016067505)]

In [46]:
model.similar_by_word('3d')

[(u'2d', 0.9850590825080872),
 (u'3dsmax', 0.9630557894706726),
 (u'rendering', 0.9622738361358643),
 (u'3ds_max', 0.9596410393714905),
 (u'vray', 0.9593963623046875),
 (u'terrain', 0.9567844867706299),
 (u'weapons', 0.9553167819976807),
 (u'autodesk_3ds_max', 0.9520688056945801),
 (u'zbrush', 0.9519199728965759),
 (u'3d_artist', 0.9511045813560486)]

In [47]:
model.similar_by_word('wordpress')

[(u'drupal', 0.968530535697937),
 (u'joomla', 0.9621365070343018),
 (u'oscommerce', 0.9494010806083679),
 (u'magento', 0.9465824961662292),
 (u'core_php', 0.9465187788009644),
 (u'cake_php', 0.941646933555603),
 (u'opencart', 0.9410149455070496),
 (u'zend_framework', 0.9374854564666748),
 (u'smarty', 0.9365031719207764),
 (u'cakephp', 0.9353802800178528)]

In [48]:
model.similar_by_word('jquery')

[(u'ajax', 0.9046893119812012),
 (u'html5', 0.886608898639679),
 (u'css3', 0.8828490972518921),
 (u'web_development', 0.8801085948944092),
 (u'mvc', 0.8758147954940796),
 (u'dhtml', 0.875282883644104),
 (u'xhtml', 0.8744614124298096),
 (u'html_5', 0.8666660785675049),
 (u'mvc_framework', 0.8644943237304688),
 (u'ui_development', 0.851926326751709)]

In [49]:
model.similar_by_word('ajax')

[(u'dhtml', 0.9191069602966309),
 (u'xhtml', 0.9145079851150513),
 (u'css3', 0.9141147136688232),
 (u'mvc', 0.9137628674507141),
 (u'web_development', 0.9129828214645386),
 (u'mvc_framework', 0.9075711965560913),
 (u'oops_concept', 0.9058328866958618),
 (u'html_5', 0.9055528044700623),
 (u'jquery', 0.9046891331672668),
 (u'dojo', 0.9034239649772644)]